# Demo

In [ ]:
!pip install tai-chi-engine

## Probable other dependencies
transformers==4.12.3

In [1]:
from forgebox.imports import *
from tai_chi_engine import TaiChiEngine

## Demo tasks

> Load all the code above in one shot, the demo starts here

In [2]:
def df_creator_image_folder(path: Path) -> pd.DataFrame:
    """
    Create a dataframe ,
    Which list all the image path under a system folder
    """
    path = Path(path)
    files = []
    formats = ["jpg", "jpeg", "png"]
    for fmt in formats:
        files.extend(path.rglob(f"*.{fmt.lower()}"))
        files.extend(path.rglob(f"*.{fmt.upper()}"))
    return pd.DataFrame({"path": files}).sample(frac=1.).reset_index(drop=True)

### Choose dataset

In [7]:
# BEAR_DATASET = HOME/"Downloads"/"bear_dataset"
DATA = Path("/GCI/data")
BEAR_DATASET = DATA/"bear_dataset"
ROTTEN_TOMATOES = DATA/"rttmt"
NETFLIX = DATA/"nf"

Choose one of the following to run 

#### Netflix 📺

In [17]:
base_df = pd.read_csv(NETFLIX/"netflix_titles.csv")
base_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


#### The bear 🐻

In [10]:
base_df = df_creator_image_folder(BEAR_DATASET)
base_df

,path
0,/GCI/data/bear_dataset/teddys/00000100.jpg
1,/GCI/data/bear_dataset/grizzly/00000190.jpg
2,/GCI/data/bear_dataset/teddys/00000076.jpg
3,/GCI/data/bear_dataset/teddys/00000011.jpg
4,/GCI/data/bear_dataset/teddys/00000090.jpg
...,...
517,/GCI/data/bear_dataset/teddys/00000158.jpg
518,/GCI/data/bear_dataset/black/00000112.JPG
519,/GCI/data/bear_dataset/grizzly/00000142.jpg
520,/GCI/data/bear_dataset/grizzly/00000075.jpg


#### The rotten tomatoes 🍅 🎬

In [19]:
# the rotten tomatoes dataset, we are not using every line

base_df = pd.read_csv(ROTTEN_TOMATOES/'critic_reviews.csv', nrows=200000)
base_df = base_df[~base_df['review_score'].isna()].reset_index(drop=True)
base_df = base_df[~base_df['review_content'].isna()].reset_index(drop=True)
base_df = base_df[~base_df['critic_name'].isna()].reset_index(drop=True)

base_df = base_df[base_df['review_score'].apply(lambda x: "/" in x)].reset_index(drop=True)

base_df['review_score'] = base_df['review_score'].apply(eval)

base_df

### Start of the pipeline

Initiate the ```phase``` to track the configuration

In [11]:
PROJECT = Path("./project")
# PROJECT = Path("./project/image_regression")
# PROJECT = Path("./project/rotten1")
# PROJECT = Path("./project/rotten_text")
# PROJECT = Path("./project/netflix")

engine = TaiChiEngine(base_df, project = PROJECT)

In [12]:
engine()

Output()